In [2]:
import pandas as pd
import requests
from tqdm import tqdm
import time
import re


# Bueno

In [5]:
import pandas as pd
import requests
import time
import random
import html
import logging
from tqdm import tqdm

# ===========================
# 1. CONFIGURACIÓN DEL LOGGER
# ===========================
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        # logging.FileHandler("wikidata_etl.log")  # Descomenta si quieres guardar en archivo
    ]
)
logger = logging.getLogger(__name__)

# =======================
# 2. CARGAR Y LIMPIAR CSV
# =======================
logger.info("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])
artistas_unicos = df['raw'].dropna().unique()

# =======================
# 3. FUNCIONES DE APOYO
# =======================
def limpiar_nombre(nombre):
    nombre = str(nombre)
    nombre = nombre.replace('"', '\\"')
    nombre = nombre.replace("\n", " ").strip()
    return html.escape(nombre)

def construir_query_sparql(artistas):
    values = "\n".join([f'"{limpiar_nombre(nombre)}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?death ?countryLabel ?awardLabel ?genderLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      ?artist wdt:P166 ?award.
      ?award rdfs:label ?awardLabel.
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    return query

# =======================
# 4. CONSULTA ROBUSTA
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
HEADERS = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "ETL-Musical-Artists/1.0 (tucorreo@ejemplo.com)"
}

def obtener_datos_wikidata(artistas_batch, max_retries=4):
    query = construir_query_sparql(artistas_batch)
    for intento in range(max_retries):
        try:
            response = requests.post(
                WIKIDATA_ENDPOINT,
                headers=HEADERS,
                data={"query": query},
                timeout=30
            )
            if response.status_code == 429:
                wait = 2 ** intento + random.uniform(0, 0.5)
                logger.warning(f"⚠️  429 Too Many Requests. Esperando {wait:.1f}s...")
                time.sleep(wait)
                continue
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            wait = 2 ** intento + random.uniform(0, 0.5)
            logger.error(f"❌ Error intento {intento + 1}: {e} → Reintentando en {wait:.1f}s...")
            time.sleep(wait)
    logger.critical("🚫 Fallo final tras múltiples reintentos.")
    return None

# =======================
# 5. CONSULTA POR BATCHES
# =======================
logger.info("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 60
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)

            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown")
                    })
                logger.debug(f"✅ Batch exitoso en índice {i} (size={batch_size})")
                i += batch_size
                pbar.update(batch_size)
                batch_success = True
            else:
                batch_size = batch_size // 2

        if not batch_success:
            logger.warning(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 6. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "award", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)
#df_result.to_csv("../data/api_data.csv", index=False)
logger.info("✅ Datos guardados en '../data/api_data.csv'")


2025-04-09 21:39:28,794 [INFO] 🔄 Cargando y limpiando datos...
2025-04-09 21:39:28,821 [INFO] 🚀 Consultando Wikidata...
🔎 Batches SPARQL:   0%|          | 60/29858 [00:14<1:57:06,  4.24it/s]2025-04-09 21:39:43,211 [ERROR] ❌ Error intento 1: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql → Reintentando en 1.0s...
2025-04-09 21:39:44,480 [ERROR] ❌ Error intento 2: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql → Reintentando en 2.2s...
2025-04-09 21:39:46,926 [ERROR] ❌ Error intento 3: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql → Reintentando en 4.5s...
2025-04-09 21:39:51,778 [ERROR] ❌ Error intento 4: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql → Reintentando en 8.4s...
🔎 Batches SPARQL:   0%|          | 60/29858 [00:31<4:19:12,  1.92it/s]


KeyboardInterrupt: 

In [6]:

# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
print("🔄 Cargando y limpiando datos...")

df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])
artistas_unicos = df['raw'].unique()

# =======================
# 2. CONSULTA SPARQL CON TODOS LOS CAMPOS
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"

def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?death ?countryLabel ?awardLabel ?genderLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      ?artist wdt:P166 ?award.
      ?award rdfs:label ?awardLabel.
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}  # Género con P21
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    headers = {"Accept": "application/sparql-results+json"}
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 60
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown")  # Valor por defecto si no hay género
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(1)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "award", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)



🔄 Cargando y limpiando datos...
🚀 Consultando Wikidata...


🔎 Batches SPARQL:   0%|          | 60/29859 [00:01<10:28, 47.44it/s]

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql


🔎 Batches SPARQL:   0%|          | 90/29859 [00:06<43:39, 11.37it/s]

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql
❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql


...

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql
❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql
❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql
❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql


🔎 Batches SPARQL:   0%|          | 108/29859 [00:11<1:12:10,  6.87it/s]

❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql
❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql
❌ Error en SPARQL: 400 Client Error: Bad Request for url: https://query.wikidata.org/sparql


🔎 Batches SPARQL:   0%|          | 108/29859 [00:13<1:02:42,  7.91it/s]


KeyboardInterrupt: 

In [ ]:
# =======================
# 1. CARGAR Y LIMPIAR CSV
# =======================
# Función para limpiar nombres
import pandas as pd
import re

def limpiar_nombre(nombre):
    if pd.isna(nombre) or not nombre.strip():
        return None
    # Eliminar caracteres problemáticos para SPARQL
    nombre = nombre.replace("\\", "")   # Quitar backslashes
    nombre = nombre.replace('"', '')    # Quitar comillas dobles
    nombre = nombre.replace("'", "")    # Quitar comillas simples
    nombre = nombre.replace("/", " ")   # Evitar errores con `/`
    nombre = nombre.replace("&", "and") # Evitar ambigüedad
    nombre = nombre.strip()
    return nombre

# Cargar y limpiar
df = pd.read_csv("../data/artists.csv", header=None, names=["raw"])
nombres_limpios = [limpiar_nombre(nombre) for nombre in df["raw"]]
artistas_unicos = sorted(set([nombre for nombre in nombres_limpios if nombre]))

print(f"✅ Total artistas únicos: {len(artistas_unicos)}")


# =======================
# 2. CONSULTA SPARQL CON TODOS LOS CAMPOS
# =======================
WIKIDATA_ENDPOINT = "https://query.wikidata.org/sparql"
HEADERS = {
    "Accept": "application/sparql-results+json",
    "User-Agent": "ETL-MusicalProject/1.0 (malvadocucarachon@gmail.com)"  # <-- ¡Pon tu info real aquí!
}


def construir_query_sparql(artistas):
    values = "\n".join([f'"{nombre}"@en' for nombre in artistas])
    query = f"""
    SELECT ?artistLabel ?death ?countryLabel ?awardLabel ?genderLabel WHERE {{
      VALUES ?name {{ {values} }}
      ?artist rdfs:label ?name.
      ?artist wdt:P166 ?award.
      ?award rdfs:label ?awardLabel.
      OPTIONAL {{ ?artist wdt:P27 ?country. }}
      OPTIONAL {{ ?artist wdt:P570 ?death. }}
      OPTIONAL {{ ?artist wdt:P21 ?gender. }}  # Género con P21
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }}
    }}
    """
    return query

def obtener_datos_wikidata(artistas_batch):
    query = construir_query_sparql(artistas_batch)
    try:
        response = requests.post(WIKIDATA_ENDPOINT, data={"query": query}, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print("❌ Error en SPARQL:", e)
        return None

# =======================
# 3. CONSULTA ROBUSTA
# =======================
print("🚀 Consultando Wikidata...")

MAX_QUERY_SIZE = 60000
results = []
i = 0

with tqdm(total=len(artistas_unicos), desc="🔎 Batches SPARQL") as pbar:
    while i < len(artistas_unicos):
        batch_size = 80
        batch_success = False

        while batch_size > 0 and not batch_success:
            batch = artistas_unicos[i:i+batch_size]
            query = construir_query_sparql(batch)
            if len(query.encode("utf-8")) > MAX_QUERY_SIZE:
                batch_size -= 5
                continue

            data = obtener_datos_wikidata(batch)
            if data:
                for row in data["results"]["bindings"]:
                    results.append({
                        "artist": row.get("artistLabel", {}).get("value", ""),
                        "death": row.get("death", {}).get("value", ""),
                        "country": row.get("countryLabel", {}).get("value", ""),
                        "award": row.get("awardLabel", {}).get("value", "No awards"),
                        "gender": row.get("genderLabel", {}).get("value", "Unknown")  # Valor por defecto si no hay género
                    })
                batch_success = True
                i += batch_size
                pbar.update(batch_size)
                time.sleep(0.8)
            else:
                batch_size = batch_size // 2

        if not batch_success:
            print(f"⚠️  Saltando artista en índice {i}: {artistas_unicos[i]}")
            i += 1
            pbar.update(1)

# =======================
# 4. GUARDAR RESULTADOS
# =======================
columnas_ordenadas = ["artist", "country", "award", "death", "gender"]
df_result = pd.DataFrame(results, columns=columnas_ordenadas)

✅ Total artistas únicos: 29856
🚀 Consultando Wikidata...


🔎 Batches SPARQL: 29920it [14:01, 35.56it/s]                           


In [18]:
# Calcular el punto de división (por la mitad en este caso)
halfway = len(df_result) // 2

# Guardar la primera parte del DataFrame
df_result.iloc[:halfway].to_csv("../data/wikidata_artists_part1.csv", index=False)
print("✅ Archivo 'wikidata_artists_part1.csv' creado correctamente.")

# Guardar la segunda parte del DataFrame
df_result.iloc[halfway:].to_csv("../data/wikidata_artists_part2.csv", index=False)
print("✅ Archivo 'wikidata_artists_part2.csv' creado correctamente.")

✅ Archivo 'wikidata_artists_part1.csv' creado correctamente.
✅ Archivo 'wikidata_artists_part2.csv' creado correctamente.
